# Quick start

#### [Open](https://colab.research.google.com/github/Bihaqo/t3f/blob/develop/docs/quick_start.ipynb) this page in an interactive mode via Google Colaboratory.

In this quick starting guide we show the basics of working with t3f library. The main concept of the library is a TensorTrain object -- a compact (factorized) representation of a tensor (=multidimensional array). This is generalization of the matrix low-rank decomposition.


To begin, [install T3F](https://t3f.readthedocs.io/en/latest/installation.html), import some libraries, and enable [eager execution mode](https://www.tensorflow.org/guide/eager) which simplifies workflow with TensorFlow

In [1]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
try:
    import t3f
except ImportError:
    # Install T3F if it's not already installed.
    !git clone https://github.com/Bihaqo/t3f.git
    !cd t3f; pip install .
    import t3f

Let's start with converting a dense (numpy) matrix into the TT-format, which in this case coinsides with the low-rank format.

In [2]:
# Generate a random dense matrix of size 3 x 4.
a_dense = np.random.randn(3, 4)
# Convert the matrix into the TT-format with TT-rank = 3 (the larger the TT-rank, the more exactly the tensor will be converted, but the more memory and time everything will take). For matrices, matrix rank coinsides with TT-rank.
a_tt = t3f.to_tt_tensor(a_dense, max_tt_rank=3)
# a_tt stores the factorized representation of the matrix, namely it stores the matrix as a product of two smaller matrices which are called TT-cores. You can access the TT-cores directly.
print(a_tt.tt_cores)
# To check that the convertions into the TT-format didn't change the matrix too much, let's convert it back and compare to the original.
reconstructed_matrix = t3f.full(a_tt)
print(reconstructed_matrix, a_dense)


TypeError: unhashable type: 'list'

The same idea applies to tensors

In [ ]:
# Generate a random dense tensor of size 3 x 2 x 2.
a_dense = np.random.randn(3, 2, 2)
# Convert the tensor into the TT-format with TT-rank = 3.
a_tt = t3f.to_tt_tensor(a_dense, max_tt_rank=3)
# Print TT-cores that compactly represent the tensor.
print(a_tt.tt_cores)
# To check that the convertions into the TT-format didn't change the tensor too much, let's convert it back and compare to the original.
reconstructed_tensor = t3f.full(a_tt)
print(reconstructed_tensor, a_dense)


T3F is a library of different operations that can be applied to the tensors in the TT-format (by working directly with the compact representation, i.e. without the need to materialize the tensors themself).
Here are some basic examples

In [ ]:
# Create a random tensor of shape (3, 2, 2) directly in the TT-format (in contrast to generating a dense tensor and then converting it to TT).
b_tt = t3f.random_tensor((3, 2, 2), tt_rank=3)
# Compute the Frobenius norm of the tensor.
norm = t3f.frobenius_norm(b_tt)
print(norm)
# Compute the TT-representation of the sum or elementwise product of two TT-tensors.
sum = a_tt + b_tt
prod = a_tt * b_tt
twice_a = 2 * a_tt
# Most operations on TT-tensors increase the TT-rank. After applying a seqeunce of operations the TT-rank can increase by too much and we may want to reduce it. To do that there is a rounding operation, which finds the closes tensor to the given one of a smaller rank.
rounded_prod = t3f.round(prod)
CHECK RANKS!
